# Perform on Colab

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
downloaded = drive.CreateFile({'id':'1vU9exH1I4zObuOzihDgm7v9Ubq4rCSoq'}) # replace the id with id of file you want to access
downloaded.GetContentFile('WheatHeadDetection') 

In [ ]:
from zipfile import ZipFile

In [ ]:
file_name = 'WheatHeadDetection'
with ZipFile(file_name , 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
!git clone https://github.com/fizyr/keras-retinanet.git

Cloning into 'keras-retinanet'...
remote: Enumerating objects: 5862, done.
remote: Total 5862 (delta 0), reused 0 (delta 0), pack-reused 5862
Receiving objects: 100% (5862/5862), 13.40 MiB | 10.44 MiB/s, done.
Resolving deltas: 100% (3948/3948), done.


In [ ]:
%cd keras-retinanet/

!pip install .

/content/keras-retinanet
Processing /content/keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp36-cp36m-linux_x86_64.whl size=170769 sha256=ff419269c657fbcce4953b101a13a34299806a7fa9c1f090da0d60ae0c689e27
  Stored in directory: /root/.cache/pip/wheels/b2/9f/57/cb0305f6f5a41fc3c11ad67b8cedfbe9127775b563337827ba
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13346 sha256=5cc60982c76cda3cb462d75e9af20c2a10976c3ac869918366b1d293e6ad63bb
  Stored in directory: /root/.cache/pip/wheels/80/dd/ac/842235b63dddac12faa4b48ebe58b8944e8c2e57c2e38dddb6
Successfully built keras-retinanet keras-resnet


In [ ]:
!pip install gdown


In [ ]:
!nvidia-smi

Sun Jul  5 12:05:24 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python setup.py build_ext --inplace

running build_ext
cythoning keras_retinanet/utils/compute_overlap.pyx to keras_retinanet/utils/compute_overlap.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/keras-retinanet/keras_retinanet/utils/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'keras_retinanet.utils.compute_overlap' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/keras_retinanet
creating build/temp.linux-x86_64-3.6/keras_retinanet/utils
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -c keras_retinanet/utils/compute_overlap.c -o build/temp.linux-x86_64-3.6/keras_retinanet/utils/compute_over

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 516.2MB 17kB/s 


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import pandas as pd
data = pd.read_csv(r'/content/train.csv')

In [ ]:
len(data)

147793

In [ ]:
image = []
for i in data['image_id']:
    j = str('/content/train/'+i+'.jpg')
    image.append(j)

In [ ]:
data.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [ ]:
bboxes = {}
xmin = []
ymin = []
width = []
height = []
for i, image_id in enumerate(data['image_id']):
    bbox = data['bbox'][i][1:-1]
    bbox = bbox.split(',')
    xmin.append(float(bbox[0]))
    ymin.append(float(bbox[1]))
    width.append(float(bbox[2]))
    height.append(float(bbox[3]))
    
    if image_id not in bboxes:
        bboxes[image_id] = []
        bboxes[image_id].append([xmin, ymin, width, height])
    else:
        bboxes[image_id].append([xmin, ymin, width, height])

In [ ]:
data['xmin'] = xmin
data['ymin'] = ymin
data['xmax'] = (width + data['xmin']).astype(int)
data['ymax'] = (height + data['ymin']).astype(int)

data['xmin'] = data['xmin'].astype(int)
data['ymin'] = data['ymin'].astype(int)


data['image'] = image

data = data.drop(['bbox' , 'bbox' , 'width' , 'height' , 'image_id'] , axis = 1)

data['class_name'] = 'Wheat_Heads'

In [ ]:
data.head()

,source,xmin,ymin,xmax,ymax,image,class_name
0,usask_1,834,222,890,258,/content/train/b6ab77fd7.jpg,Wheat_Heads
1,usask_1,226,548,356,606,/content/train/b6ab77fd7.jpg,Wheat_Heads
2,usask_1,377,504,451,664,/content/train/b6ab77fd7.jpg,Wheat_Heads
3,usask_1,834,95,943,202,/content/train/b6ab77fd7.jpg,Wheat_Heads
4,usask_1,26,144,150,261,/content/train/b6ab77fd7.jpg,Wheat_Heads


In [ ]:
data = data.drop(['source'] , axis = 1)

In [ ]:
data = data[[ 'image' , 'xmin' , 'ymin' , 'xmax' , 'ymax' , 'class_name']]

In [ ]:
data.head()

,image,xmin,ymin,xmax,ymax,class_name
0,/content/train/b6ab77fd7.jpg,834,222,890,258,Wheat_Heads
1,/content/train/b6ab77fd7.jpg,226,548,356,606,Wheat_Heads
2,/content/train/b6ab77fd7.jpg,377,504,451,664,Wheat_Heads
3,/content/train/b6ab77fd7.jpg,834,95,943,202,Wheat_Heads
4,/content/train/b6ab77fd7.jpg,26,144,150,261,Wheat_Heads


In [ ]:
train_data, test_data = train_test_split(
  data, 
  test_size=0.2, 
  shuffle=False
)

In [ ]:

ANNOTATIONS_FILE = 'annotations.csv'
CLASSES_FILE = 'classes.csv'

train_data.to_csv(ANNOTATIONS_FILE, index=False, header=None)

classes = set(['Wheat_Heads'])

with open(CLASSES_FILE, 'w') as f:
    for i, line in enumerate(sorted(classes)):
        f.write('{},{}\n'.format(line,i))

In [ ]:
!head classes.csv

Wheat_Heads,0


In [ ]:
!head annotations.csv

/content/train/b6ab77fd7.jpg,834,222,890,258,Wheat_Heads
/content/train/b6ab77fd7.jpg,226,548,356,606,Wheat_Heads
/content/train/b6ab77fd7.jpg,377,504,451,664,Wheat_Heads
/content/train/b6ab77fd7.jpg,834,95,943,202,Wheat_Heads
/content/train/b6ab77fd7.jpg,26,144,150,261,Wheat_Heads
/content/train/b6ab77fd7.jpg,569,382,688,493,Wheat_Heads
/content/train/b6ab77fd7.jpg,52,602,134,647,Wheat_Heads
/content/train/b6ab77fd7.jpg,627,302,749,377,Wheat_Heads
/content/train/b6ab77fd7.jpg,412,367,480,449,Wheat_Heads
/content/train/b6ab77fd7.jpg,953,220,1009,323,Wheat_Heads


In [ ]:
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'

URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

print('Downloaded pretrained model to ' + PRETRAINED_MODEL)

Downloaded pretrained model to ./snapshots/_pretrained_model.h5


In [ ]:
!keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 8 --steps 500 --epochs 10 csv annotations.csv classes.csv

Using TensorFlow backend.
Creating model, this may take a second...
2020-07-05 12:11:13.124586: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-05 12:11:13.156164: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-05 12:11:13.156770: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-07-05 12:11:13.157062: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-07-05 12:11:13.158794: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-07